In [104]:
import glob, os
from astropy.io import ascii
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, vstack
import matplotlib as mpl
from astropy import units as u
import time
from astropy.stats import sigma_clip
import bottleneck as bn

In [105]:
import matplotlib as mpl
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

mpl.rcParams["axes.titlesize"] = 14
mpl.rcParams["axes.labelsize"] = 20
plt.rcParams['savefig.dpi'] = 200
plt.rc('font', family='serif')

In [106]:
zimlist = sorted(glob.glob('../loao/C*0.fits'))
zimlist[:5]

['../loao/Calib-LOAO-GRB221009A-20221010-040621-y-60.fits',
 '../loao/Calib-LOAO-GRB221009A-20221010-041539-y-60.fits',
 '../loao/Calib-LOAO-GRB221009A-20221010-042549-y-60.fits',
 '../loao/Calib-LOAO-GRB221009A-20221010-043625-y-60.fits',
 '../loao/Calib-LOAO-GRB221009A-20221010-044539-y-60.fits']

In [107]:
dateobslist = sorted(list(set([os.path.basename(inim).split('-')[3] for inim in zimlist])))
dateobslist

['20221010', '20221011', '20221012', '20221013', '20221014']

In [108]:
#	Part
ii = 0
dateobs = dateobslist[ii]
pimlist = sorted(glob.glob(f'../loao/Calib-*{dateobs}*0.fits'))
pimlist

['../loao/Calib-LOAO-GRB221009A-20221010-040621-y-60.fits',
 '../loao/Calib-LOAO-GRB221009A-20221010-041539-y-60.fits',
 '../loao/Calib-LOAO-GRB221009A-20221010-042549-y-60.fits',
 '../loao/Calib-LOAO-GRB221009A-20221010-043625-y-60.fits',
 '../loao/Calib-LOAO-GRB221009A-20221010-044539-y-60.fits']

In [109]:
datalist = []
for inim in zimlist:
	data, hdr = fits.getdata(inim, header=True)
	bkg = np.median(sigma_clip(data.flatten(), sigma=3, maxiters=5, cenfunc=bn.nanmedian, masked=False, copy=False))
	datalist.append(data/bkg)
mdata = np.median(datalist, axis=0)
mbkg = bn.nanmedian(mdata.flatten())

In [110]:
for inim in zimlist:
	#	Single image
	data, hdr = fits.getdata(inim, header=True)
	bkg = np.median(sigma_clip(data.flatten(), sigma=3, maxiters=5, cenfunc=bn.nanmedian, masked=False, copy=False))

	dfdata = data-mdata*(bkg/mbkg)
	dfbkg = bn.nanmedian(dfdata.flatten())
	fits.writeto(f"../loao/df{os.path.basename(inim)}", dfdata, header=hdr, overwrite=True)

/var/folders/73/438czwkx0jb_7z61gc79vcb40000gn/T/ipykernel_44752/3799743330.py:6: RuntimeWarning: invalid value encountered in subtract
  dfdata = data-mdata*(bkg/mbkg)
/var/folders/73/438czwkx0jb_7z61gc79vcb40000gn/T/ipykernel_44752/3799743330.py:6: RuntimeWarning: invalid value encountered in subtract
  dfdata = data-mdata*(bkg/mbkg)
/var/folders/73/438czwkx0jb_7z61gc79vcb40000gn/T/ipykernel_44752/3799743330.py:6: RuntimeWarning: invalid value encountered in subtract
  dfdata = data-mdata*(bkg/mbkg)
/var/folders/73/438czwkx0jb_7z61gc79vcb40000gn/T/ipykernel_44752/3799743330.py:6: RuntimeWarning: invalid value encountered in subtract
  dfdata = data-mdata*(bkg/mbkg)
/var/folders/73/438czwkx0jb_7z61gc79vcb40000gn/T/ipykernel_44752/3799743330.py:6: RuntimeWarning: invalid value encountered in subtract
  dfdata = data-mdata*(bkg/mbkg)
/var/folders/73/438czwkx0jb_7z61gc79vcb40000gn/T/ipykernel_44752/3799743330.py:6: RuntimeWarning: invalid value encountered in subtract
  dfdata = data-mda

In [111]:
fig = plt.figure(figsize=(18, 6))
plt.subplot(131)
plt.imshow(data, vmin=bkg*1, vmax=bkg*1.05)
plt.subplot(132)
plt.imshow(mdata, vmin=1, vmax=1.05)
plt.subplot(133)
plt.imshow(dfdata, vmin=dfbkg, vmax=dfbkg*1.05)

Error in callback <function _draw_all_if_interactive at 0x11f971480> (for post_execute):


ValueError: minvalue must be less than or equal to maxvalue

ValueError: minvalue must be less than or equal to maxvalue

<Figure size 1800x600 with 3 Axes>